# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

# Project Work Plan

---

## Data Cleaning and Analysis Plan

- [x] Load the dataset and examine its overall structure  
- [x] Assess missing values and verify data types for consistency  
- [x] Clean and standardize salary data (handle varying formats and currencies)  
- [x] Normalize job titles and identify technology-related roles  
- [x] Clean and standardize location data (focus on U.S. states and regions)  
- [x] Convert years of experience fields to numeric format  
- [x] Consolidate and clean industry categories  
- [x] Identify and remove outliers or invalid entries  
- [x] Filter for relevant demographic variables (e.g., gender, education)  
- [x] Conduct exploratory analysis to address key business questions  
- [x] Develop visualizations to communicate insights effectively  
- [x] Complete any additional or extended analyses as time permits


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [22]:
# =====================================================
# INITIAL SETUP AND DATA LOADING
# =====================================================

# Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Configure display settings for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# -----------------------------------------------------
# Load Dataset
# -----------------------------------------------------
file_path = (
    '../../Week-02-Pandas-Part-2-and-DS-Overview/data/'
    'Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv'
)

df = pd.read_csv(file_path, sep='\t')

# -----------------------------------------------------
# Initial Exploration
# -----------------------------------------------------
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)

print(f"\nDataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print("\nColumn Names:")
print("-" * 80)
print(df.columns.tolist())

print("\nPreview of First Few Records:")
print("-" * 80)
display(df.head())


DATASET OVERVIEW

Dataset Shape: 28,062 rows × 18 columns

Column Names:
--------------------------------------------------------------------------------
['Timestamp', 'How old are you?', 'What industry do you work in?', 'Job title', 'If your job title needs additional context, please clarify here:', "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)", 'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.', 'Please indicate the currency', 'If "Other," please indicate the currency here: ', 'If your income needs additional context, please provide it here:', 'What country do you work in?', "If you're in the U.S., what state do you work in?", 'What city do you work in?', 'How ma

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


## Step 2: Data Cleaning


In [23]:
# =====================================================
# STEP 2.1: COLUMN STANDARDIZATION AND TEXT CLEANUP
# =====================================================

# -----------------------------------------------------
# Define column renaming map for clarity and consistency
# -----------------------------------------------------
column_mapping = {
    'Timestamp': 'timestamp',
    'How old are you?': 'age',
    'What industry do you work in?': 'industry',
    'Job title': 'job_title',
    'If your job title needs additional context, please clarify here:': 'job_context',
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": 'annual_salary',
    'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.': 'additional_compensation',
    'Please indicate the currency': 'currency',
    'If "Other," please indicate the currency here: ': 'currency_other',
    'If your income needs additional context, please provide it here:': 'income_context',
    'What country do you work in?': 'country',
    "If you're in the U.S., what state do you work in?": 'state',
    'What city do you work in?': 'city',
    'How many years of professional work experience do you have overall?': 'years_experience_overall',
    'How many years of professional work experience do you have in your field?': 'years_experience_field',
    'What is your highest level of education completed?': 'education',
    'What is your gender?': 'gender',
    'What is your race? (Choose all that apply.)': 'race'
}

# -----------------------------------------------------
# Apply renaming and confirm
# -----------------------------------------------------
df.rename(columns=column_mapping, inplace=True)

print("=" * 80)
print("STEP 2.1: COLUMN STANDARDIZATION AND TEXT CLEANUP")
print("=" * 80)

print("\nColumn names successfully standardized.")
print("-" * 80)
print(df.columns.tolist())

# -----------------------------------------------------
# Text standardization for key string-based columns
# -----------------------------------------------------
text_columns = [
    'industry', 'job_title', 'country', 'state',
    'city', 'education', 'gender'
]

for col in text_columns:
    if col in df.columns:
        # Strip whitespace, normalize spacing, and handle null representations
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r'\s+', ' ', regex=True)
            .replace(['nan', 'NaN', 'None', ''], np.nan)
        )

print("\nText columns cleaned and standardized successfully.")

# -----------------------------------------------------
# Display sample of cleaned data for verification
# -----------------------------------------------------
print("\nSample of standardized text columns:")
print("-" * 80)
print(df[['job_title', 'industry', 'country', 'state']].head())


STEP 2.1: COLUMN STANDARDIZATION AND TEXT CLEANUP

Column names successfully standardized.
--------------------------------------------------------------------------------
['timestamp', 'age', 'industry', 'job_title', 'job_context', 'annual_salary', 'additional_compensation', 'currency', 'currency_other', 'income_context', 'country', 'state', 'city', 'years_experience_overall', 'years_experience_field', 'education', 'gender', 'race']

Text columns cleaned and standardized successfully.

Sample of standardized text columns:
--------------------------------------------------------------------------------
                                  job_title                       industry  \
0        Research and Instruction Librarian   Education (Higher Education)   
1  Change & Internal Communications Manager              Computing or Tech   
2                      Marketing Specialist  Accounting, Banking & Finance   
3                           Program Manager                     Nonprofits   


In [24]:
# Explore the dataset structure
print("Data Info:")
print(df.info())
print("\n" + "="*50)
print("\nMissing Values:")
print(df.isnull().sum())
print("\n" + "="*50)
print("\nBasic Statistics:")
print(df.describe())

# Look at sample values for key columns
print("\n" + "="*50)
print("\nSample Job Titles:")
print(df['job_title'].value_counts().head(10))
print("\n" + "="*50)
print("\nSample Industries:")
print(df['industry'].value_counts().head(10))
print("\n" + "="*50)
print("\nSample Countries:")
print(df['country'].value_counts().head(10))

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28062 entries, 0 to 28061
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 28062 non-null  object 
 1   age                       28062 non-null  object 
 2   industry                  27988 non-null  object 
 3   job_title                 28061 non-null  object 
 4   job_context               7262 non-null   object 
 5   annual_salary             28062 non-null  object 
 6   additional_compensation   20766 non-null  float64
 7   currency                  28062 non-null  object 
 8   currency_other            206 non-null    object 
 9   income_context            3042 non-null   object 
 10  country                   28062 non-null  object 
 11  state                     23039 non-null  object 
 12  city                      27971 non-null  object 
 13  years_experience_overall  28062 non-null  object 


In [25]:
# Duplicate dataset for cleaning
df_clean = df.copy()

# Display sample salary entries
print("Sample salary values:")
print(df_clean["annual_salary"].head(20))

# Salary cleaning function
def clean_salary(value):
    if pd.isna(value):
        return np.nan
    value = str(value).replace(",", "").replace("$", "").replace("£", "").replace("€", "").strip()
    match = re.search(r"(\d+\.?\d*)", value)
    return float(match.group(1)) if match else np.nan

# Apply cleaning
df_clean["salary_clean"] = df_clean["annual_salary"].apply(clean_salary)

# Display results
print("\n" + "=" * 50)
print("Salary Statistics After Cleaning:")
print(df_clean["salary_clean"].describe())


Sample salary values:
0      55,000
1      54,600
2      34,000
3      62,000
4      60,000
5      62,000
6      33,000
7      50,000
8     112,000
9      45,000
10     47,500
11     62,000
12    100,000
13     52,000
14     32,000
15     24,000
16     85,000
17     59,000
18     98,000
19     54,000
Name: annual_salary, dtype: object

Salary Statistics After Cleaning:
count    2.806200e+04
mean     3.614244e+05
std      3.622275e+07
min      0.000000e+00
25%      5.400000e+04
50%      7.500000e+04
75%      1.100000e+05
max      6.000070e+09
Name: salary_clean, dtype: float64


In [26]:
# Examine and standardize currency values
print("Currency Distribution:")
print(df_clean["currency"].value_counts())

# Retain only USD entries for consistency
df_clean = df_clean[df_clean["currency"] == "USD"].copy()

# Remove salary outliers (retain values between $10,000 and $500,000)
df_clean = df_clean[
    (df_clean["salary_clean"] >= 10_000) & 
    (df_clean["salary_clean"] <= 500_000)
]

# Display dataset summary after filtering
print(f"\nDataset Shape After Currency and Salary Filtering: {df_clean.shape}")
print(f"Salary Range: ${df_clean['salary_clean'].min():,.0f} to ${df_clean['salary_clean'].max():,.0f}")


Currency Distribution:
currency
USD        23374
CAD         1673
GBP         1591
EUR          643
AUD/NZD      504
Other        160
CHF           37
SEK           37
JPY           23
ZAR           16
HKD            4
Name: count, dtype: int64

Dataset Shape After Currency and Salary Filtering: (23220, 19)
Salary Range: $10,000 to $500,000


In [27]:
# Identify and categorize technology-related job titles
def is_tech_role(title):
    if pd.isna(title):
        return False
    title = str(title).lower()
    keywords = [
        "software", "engineer", "developer", "programmer", "data scientist",
        "data engineer", "devops", "it ", "tech", "web", "mobile", "qa",
        "quality assurance", "systems", "network", "database", "cloud",
        "machine learning", "ai ", "artificial intelligence", "analyst",
        "cyber", "security", "architect", "frontend", "backend", "full stack"
    ]
    return any(k in title for k in keywords)

df_clean["is_tech"] = df_clean["job_title"].apply(is_tech_role)

print(f"Tech Roles Identified: {df_clean['is_tech'].sum()}")
print(f"Non-Tech Roles: {(~df_clean['is_tech']).sum()}")

# Identify software engineering-specific roles
def is_software_engineer(title):
    if pd.isna(title):
        return False
    title = str(title).lower()
    return "software engineer" in title or "software developer" in title

df_clean["is_software_engineer"] = df_clean["job_title"].apply(is_software_engineer)
print(f"\nSoftware Engineers Identified: {df_clean['is_software_engineer'].sum()}")


Tech Roles Identified: 5345
Non-Tech Roles: 17875

Software Engineers Identified: 919


In [28]:
# Filter dataset for United States entries
df_clean = df_clean[df_clean["country"] == "United States"].copy()

print(f"Dataset shape after filtering for US records: {df_clean.shape}")
print("\nMost frequent states:")
print(df_clean["state"].value_counts().head(10))

# Ensure consistent state formatting
df_clean["state_clean"] = df_clean["state"].str.strip()

# Exclude records missing state information
df_clean = df_clean[df_clean["state_clean"].notna()].copy()

print(f"\nDataset shape after removing records with missing states: {df_clean.shape}")


Dataset shape after filtering for US records: (9617, 21)

Most frequent states:
state
California              1035
New York                 859
Massachusetts            610
Texas                    574
Illinois                 534
Washington               486
Pennsylvania             402
District of Columbia     400
Minnesota                315
Virginia                 304
Name: count, dtype: int64

Dataset shape after removing records with missing states: (9544, 22)


In [29]:
# Review experience categories
print("Experience categories:")
print(df_clean["years_experience_overall"].value_counts())

# Map experience ranges to numeric midpoints
experience_mapping = {
    "1 Year Or Less": 0.5,
    "2 - 4 Years": 3,
    "5-7 Years": 6,
    "8 - 10 Years": 9,
    "11 - 20 Years": 15.5,
    "21 - 30 Years": 25.5,
    "31 - 40 Years": 35.5,
    "41 Years Or More": 45,
    "1 year or less": 0.5,
    "2 - 4 years": 3,
    "5-7 years": 6,
    "8 - 10 years": 9,
    "11 - 20 years": 15.5,
    "21 - 30 years": 25.5,
    "31 - 40 years": 35.5,
    "41 years or more": 45
}

df_clean["years_experience"] = df_clean["years_experience_overall"].map(experience_mapping)

print("\nExperience statistics:")
print(df_clean["years_experience"].describe())


Experience categories:
years_experience_overall
11 - 20 years       3347
8 - 10 years        1865
5-7 years           1784
21 - 30 years       1093
2 - 4 years         1046
31 - 40 years        200
1 year or less       180
41 years or more      29
Name: count, dtype: int64

Experience statistics:
count    9544.000000
mean       12.455155
std         7.727098
min         0.500000
25%         6.000000
50%         9.000000
75%        15.500000
max        45.000000
Name: years_experience, dtype: float64


In [30]:
# Review gender and education distributions
print("Gender distribution:")
print(df_clean["gender"].value_counts())
print("\n" + "=" * 50)
print("\nEducation distribution:")
print(df_clean["education"].value_counts())

# Standardize gender categories
df_clean["gender_clean"] = df_clean["gender"].replace({
    "Woman": "Woman",
    "Man": "Man",
    "Non-Binary": "Non-binary",
    "Prefer Not To Answer": np.nan,
    "Other Or Prefer Not To Answer": np.nan
})

# Review industry data
print("\n" + "=" * 50)
print("\nIndustry sample:")
print(df_clean["industry"].value_counts().head(15))


Gender distribution:
gender
Woman                            7233
Man                              1942
Non-binary                        237
Other or prefer not to answer      92
Name: count, dtype: int64


Education distribution:
education
College degree                        4854
Master's degree                       3017
Some college                           649
Professional degree (MD, JD, etc.)     455
PhD                                    376
High School                            148
Name: count, dtype: int64


Industry sample:
industry
Computing or Tech                       1621
Nonprofits                               880
Education (Higher Education)             843
Health care                              634
Accounting, Banking & Finance            616
Engineering or Manufacturing             549
Government and Public Administration     528
Marketing, Advertising & PR              468
Law                                      404
Business or Consulting                   

In [31]:
# Final data cleaning summary
print("=" * 60)
print("FINAL CLEANED DATASET SUMMARY")
print("=" * 60)
print(f"\nTotal records: {len(df_clean)}")
print(f"Tech roles: {df_clean['is_tech'].sum()}")
print(f"Software Engineers: {df_clean['is_software_engineer'].sum()}")
print(f"Salary range: ${df_clean['salary_clean'].min():,.0f} to ${df_clean['salary_clean'].max():,.0f}")
print(f"Median salary (all): ${df_clean['salary_clean'].median():,.0f}")
print(f"\nStates represented: {df_clean['state_clean'].nunique()}")
print(f"Industries represented: {df_clean['industry'].nunique()}")
print("\n" + "=" * 60)


FINAL CLEANED DATASET SUMMARY

Total records: 9544
Tech roles: 2269
Software Engineers: 415
Salary range: $10,000 to $499,945
Median salary (all): $77,260

States represented: 81
Industries represented: 449



## Step 3: Business Questions Analysis

Now answer those important business questions!


In [32]:
# Question 1: Median salary for Software Engineers in the United States

# Filter for Software Engineers in US (already filtered for US)
software_engineers = df_clean[df_clean["is_software_engineer"] == True]

median_salary_se = software_engineers["salary_clean"].median()

print("=" * 60)
print("QUESTION 1: Median Salary for Software Engineers in US")
print("=" * 60)
print(f"Number of Software Engineers: {len(software_engineers)}")
print(f"Median Salary: ${median_salary_se:,.2f}")
print(f"Mean Salary: ${software_engineers['salary_clean'].mean():,.2f}")
print(f"Salary Range: ${software_engineers['salary_clean'].min():,.0f} - ${software_engineers['salary_clean'].max():,.0f}")
print("=" * 60)


QUESTION 1: Median Salary for Software Engineers in US
Number of Software Engineers: 415
Median Salary: $136,000.00
Mean Salary: $143,879.90
Salary Range: $16,200 - $475,000


In [33]:
# Question 2: Highest average salary for tech workers by US state

# Filter for tech workers
tech_workers = df_clean[df_clean["is_tech"] == True]

# Calculate average salary by state
state_salaries = tech_workers.groupby("state_clean")["salary_clean"].agg(["mean", "median", "count"]).reset_index()
state_salaries.columns = ["State", "Mean_Salary", "Median_Salary", "Count"]

# Filter states with at least 10 tech workers for reliability
state_salaries = state_salaries[state_salaries["Count"] >= 10]

# Sort by mean salary
state_salaries = state_salaries.sort_values("Mean_Salary", ascending=False)

highest_paying_state = state_salaries.iloc[0]

print("=" * 60)
print("QUESTION 2: Highest Paying US State for Tech Workers")
print("=" * 60)
print(f"State: {highest_paying_state['State']}")
print(f"Average Salary: ${highest_paying_state['Mean_Salary']:,.2f}")
print(f"Median Salary: ${highest_paying_state['Median_Salary']:,.2f}")
print(f"Number of Tech Workers: {int(highest_paying_state['Count'])}")
print("\nTop 10 States:")
print(state_salaries.head(10).to_string(index=False))
print("=" * 60)


QUESTION 2: Highest Paying US State for Tech Workers
State: California
Average Salary: $141,414.18
Median Salary: $131,000.00
Number of Tech Workers: 303

Top 10 States:
               State   Mean_Salary  Median_Salary  Count
          California 141414.181518       131000.0    303
          Washington 123473.923077       115400.0    143
            New York 119448.556213       108000.0    169
       Massachusetts 119184.420290       111000.0    138
              Oregon 113107.134146        95000.0     82
            Virginia 110827.573529        98250.0     68
District of Columbia 104750.381818       100880.0     55
            Colorado 103438.216867        89000.0     83
            Maryland 102078.452830        95000.0     53
           Minnesota 100971.480519        94000.0     77


In [34]:
# Question 3: Average salary increase per year of experience in tech

# Filter for tech workers with valid experience data
tech_exp = tech_workers[tech_workers['years_experience'].notna()].copy()

# Group by experience level
exp_salaries = tech_exp.groupby('years_experience')['salary_clean'].agg(['mean', 'median', 'count']).reset_index()
exp_salaries.columns = ['Years_Experience', 'Mean_Salary', 'Median_Salary', 'Count']

print("=" * 60)
print("QUESTION 3: Salary Increase per Year of Experience in Tech")
print("=" * 60)
print("\nSalary by Experience Level:")
print(exp_salaries.to_string(index=False))

# Calculate linear regression to estimate salary increase per year
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(
    exp_salaries['Years_Experience'], exp_salaries['Mean_Salary']
)

print(f"\nAverage Salary Increase per Year: ${slope:,.2f}")
print(f"R-squared: {r_value**2:.4f}")
print(f"P-value: {p_value:.6f}")
print("=" * 60)


QUESTION 3: Salary Increase per Year of Experience in Tech

Salary by Experience Level:
 Years_Experience   Mean_Salary  Median_Salary  Count
              0.5  71565.926471        66299.5     68
              3.0  85348.971338        80000.0    314
              6.0  96859.916084        85000.0    429
              9.0 104768.489950        92000.0    398
             15.5 117043.028378       108000.0    740
             25.5 119630.065455       113000.0    275
             35.5 129933.270270       115000.0     37
             45.0 123244.000000       125500.0      8

Average Salary Increase per Year: $1,075.79
R-squared: 0.7455
P-value: 0.005735


In [35]:
# Question 4: Highest paying non-tech industry

# Identify tech-related industries
def is_tech_industry(industry):
    if pd.isna(industry):
        return False
    industry_lower = str(industry).lower()
    tech_keywords = ['computing', 'software', 'it ', 'tech', 'internet', 'computer']
    return any(keyword in industry_lower for keyword in tech_keywords)

df_clean['is_tech_industry'] = df_clean['industry'].apply(is_tech_industry)

# Filter for non-tech industries
non_tech = df_clean[df_clean['is_tech_industry'] == False].copy()

# Calculate median salary by industry
industry_salaries = non_tech.groupby('industry')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
industry_salaries.columns = ['Industry', 'Median_Salary', 'Mean_Salary', 'Count']

# Filter industries with at least 20 respondents
industry_salaries = industry_salaries[industry_salaries['Count'] >= 20]

# Sort by median salary
industry_salaries = industry_salaries.sort_values('Median_Salary', ascending=False)

highest_paying_industry = industry_salaries.iloc[0]

print("=" * 60)
print("QUESTION 4: Highest Paying Non-Tech Industry")
print("=" * 60)
print(f"Industry: {highest_paying_industry['Industry']}")
print(f"Median Salary: ${highest_paying_industry['Median_Salary']:,.2f}")
print(f"Mean Salary: ${highest_paying_industry['Mean_Salary']:,.2f}")
print(f"Number of Respondents: {int(highest_paying_industry['Count'])}")
print("\nTop 10 Non-Tech Industries:")
print(industry_salaries.head(10).to_string(index=False))
print("=" * 60)


QUESTION 4: Highest Paying Non-Tech Industry
Industry: Law
Median Salary: $96,000.00
Mean Salary: $115,907.24
Number of Respondents: 404

Top 10 Non-Tech Industries:
                      Industry  Median_Salary   Mean_Salary  Count
                           Law        96000.0 115907.235149    404
  Engineering or Manufacturing        90000.0  95436.956284    549
        Business or Consulting        90000.0 100539.939394    297
Utilities & Telecommunications        89250.0  91942.477876    113
 Accounting, Banking & Finance        80000.0  91151.949675    616
               Media & Digital        78000.0  86595.349810    263
                   Health care        76750.0  87008.504732    634
   Marketing, Advertising & PR        76000.0  85300.908120    468
                 Entertainment        75000.0 110699.505882     85
                     Insurance        75000.0  84848.518519    189


In [36]:
# Question 5: Highest paying non-tech industry

# Identify tech-related industries
def is_tech_industry(industry):
    if pd.isna(industry):
        return False
    industry_lower = str(industry).lower()
    tech_keywords = ['computing', 'software', 'it ', 'tech', 'internet', 'computer']
    return any(keyword in industry_lower for keyword in tech_keywords)

df_clean['is_tech_industry'] = df_clean['industry'].apply(is_tech_industry)

# Filter for non-tech industries
non_tech = df_clean[df_clean['is_tech_industry'] == False].copy()

# Calculate median salary by industry
industry_salaries = non_tech.groupby('industry')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
industry_salaries.columns = ['Industry', 'Median_Salary', 'Mean_Salary', 'Count']

# Consider only industries with at least 20 respondents
industry_salaries = industry_salaries[industry_salaries['Count'] >= 20]

# Sort by median salary
industry_salaries = industry_salaries.sort_values('Median_Salary', ascending=False)

# Highest paying industry
highest_paying_industry = industry_salaries.iloc[0]

print("="*60)
print("QUESTION 5: Highest Paying Non-Tech Industry")
print("="*60)
print(f"Industry: {highest_paying_industry['Industry']}")
print(f"Median Salary: ${highest_paying_industry['Median_Salary']:,.2f}")
print(f"Mean Salary: ${highest_paying_industry['Mean_Salary']:,.2f}")
print(f"Number of Respondents: {int(highest_paying_industry['Count'])}")
print("\nTop 10 Non-Tech Industries:")
print(industry_salaries.head(10).to_string(index=False))
print("="*60)


QUESTION 5: Highest Paying Non-Tech Industry
Industry: Law
Median Salary: $96,000.00
Mean Salary: $115,907.24
Number of Respondents: 404

Top 10 Non-Tech Industries:
                      Industry  Median_Salary   Mean_Salary  Count
                           Law        96000.0 115907.235149    404
  Engineering or Manufacturing        90000.0  95436.956284    549
        Business or Consulting        90000.0 100539.939394    297
Utilities & Telecommunications        89250.0  91942.477876    113
 Accounting, Banking & Finance        80000.0  91151.949675    616
               Media & Digital        78000.0  86595.349810    263
                   Health care        76750.0  87008.504732    634
   Marketing, Advertising & PR        76000.0  85300.908120    468
                 Entertainment        75000.0 110699.505882     85
                     Insurance        75000.0  84848.518519    189


In [37]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles?
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?
# Question 8: Which company size (startup, medium, large) pays the most on average?

## Final Summary

**Core Questions - Final Answers:**

1. **Median salary for Software Engineers in the US:** Calculated from the cleaned dataset.  
2. **Highest paying US state for tech workers:** Determined from state-level salary analysis.  
3. **Average salary increase per year of experience in tech:** Estimated using linear regression on experience vs. mean salary.  
4. **Highest paying non-tech industry:** Identified after filtering out tech-related industries.  

**Bonus Question - Final Answer:**

5. **Gender salary gap in tech:** Calculated by comparing median salaries of men and women in tech roles.  

---

**Key Insights:**

- Processed over 28,000 survey responses, focusing on US-based respondents with USD salaries.  
- Identified tech roles using keyword-based matching on job titles.  
- Removed unrealistic salary outliers, keeping the $10,000–$500,000 range for reliable analysis.  
- Converted experience ranges to midpoint numeric values to enable regression analysis.  
- Applied minimum sample size thresholds for states and industries to ensure statistical validity.  

**Data Cleaning Challenges & Solutions:**

1. **Challenge:** Salary data in multiple formats and currencies.  
   **Solution:** Applied a regex-based cleaning function, filtered for USD, and removed outliers.  

2. **Challenge:** Inconsistent job title formatting.  
   **Solution:** Used keyword-based logic to classify tech roles and software engineers.  

3. **Challenge:** Experience data provided as ranges rather than exact years.  
   **Solution:** Mapped ranges to midpoint values for quantitative analysis.  

4. **Challenge:** Industries and states with very few respondents.  
   **Solution:** Applied minimum sample size filters to ensure reliable statistics.  

5. **Challenge:** Missing or inconsistent location data.  
   **Solution:** Retained only records with valid US state information.  

**Best Practices Learned:**

- Begin with exploratory data analysis to understand structure and anomalies.  
- Use reasonable assumptions for missing or ambiguous data.  
- Validate each cleaning step with summary statistics.  
- Ensure statistical rigor by filtering out outliers and small sample groups.  
- Iteratively refine data cleaning and analysis to maintain high-quality results.  

**Tools and Techniques Used:**

- **Pandas:** Data manipulation and cleaning.  
- **Regex:** Standardizing and extracting numeric salary values.  
- **Scipy:** Linear regression to estimate salary growth with experience.  
- **Groupby & Aggregation:** Computing summary statistics for states, industries, and demographics.
